In [2]:
"""
准备数据集
每个人单独标准化的数据，不需要再进行normalization
"""

import joblib
import time
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
import logging

import sys
sys.path.append('..')

from utils.utils_logger import logger_init


from utils.utils_model import form_params_grid
from utils.utils_iFEMG_feature import df_save_csv

In [4]:
# 读数据
data_df = pd.read_csv(r"E:\fuckinggggg\paper_iFEMG_4th\data\fea_normed.csv", index_col = None, header = 0)
# data_df = pd.read_csv(r"E:\Data\paper2\iFEMG\iFEMG_extension_abs_normed_all.csv", index_col = None, header = 0)

print(data_df.shape)
data_df.head()

(306, 58)


,subject,side,motion_type,label,RFinitial_pressure_ave,RFFMG,RFmav,RFrms,RFwave_length,RFzero_crossing,...,SemiTmean_power_freq,VMinitial_pressure_ave,VMFMG,VMmav,VMrms,VMwave_length,VMzero_crossing,VMslope_sign_change,VMmean_freq,VMmean_power_freq
0,241128s2,affected,qian,0.0,0.752485,-0.428352,0.329448,0.681695,1.101179,1.071944,...,-1.131997,-1.273612,-1.006979,-0.744626,-0.588665,-0.726198,-0.523945,2.581633,-0.876736,-0.929566
1,241128s2,affected,qian,0.0,0.752485,-0.899854,1.047155,0.822488,0.681090,-1.635222,...,-1.133412,-1.273612,-1.436965,-0.435984,-0.035511,-0.636153,-0.927388,-0.076157,-0.786561,-0.782346
2,241128s2,affected,qian,0.0,0.752485,-0.174328,-0.005703,0.623954,0.681109,2.268440,...,-0.702750,-1.273612,-0.699362,-0.631736,-0.287867,-0.705165,-0.946102,1.493874,-0.857919,-0.924915
3,241128s2,affected,qian,0.0,0.752485,-0.367778,0.239311,0.706075,0.328863,0.785314,...,-0.650732,-1.273612,-0.434836,-0.669685,-0.474161,-0.701455,-0.321115,1.028299,-0.708094,-0.824413
4,241128s2,affected,qian,0.0,0.752485,-0.196038,0.662971,0.714514,0.649762,-0.746958,...,0.217581,-1.273612,-0.321577,-0.747155,-0.634527,-0.722887,-1.223468,1.013098,-0.809140,-0.765498


In [5]:
"""
数据清洗 remove the data bellow:
    1. NaN value
    2. label is MVC
"""
# replace 'MVC' with NaN
data_df = data_df.replace('MVC', np.nan)
# delete NaN value
data_df = data_df.dropna(how = 'any')

print(data_df.shape)
data_df.dtypes

(306, 58)


subject                       object
side                          object
motion_type                   object
label                        float64
RFinitial_pressure_ave       float64
RFFMG                        float64
RFmav                        float64
RFrms                        float64
RFwave_length                float64
RFzero_crossing              float64
RFslope_sign_change          float64
RFmean_freq                  float64
RFmean_power_freq            float64
LFinitial_pressure_ave       float64
LFFMG                        float64
LFmav                        float64
LFrms                        float64
LFwave_length                float64
LFzero_crossing              float64
LFslope_sign_change          float64
LFmean_freq                  float64
LFmean_power_freq            float64
SarTinitial_pressure_ave     float64
SarTFMG                      float64
SarTmav                      float64
SarTrms                      float64
SarTwave_length              float64
S

In [27]:
data_df['label'].values

array([0. , 0. , 0. , 0. , 0. , 0. , 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 3. ,
       3. , 3. , 3. , 3. , 3. , 0. , 0. , 0. , 0. , 0. , 0. , 1.5, 1.5,
       1.5, 1.5, 1.5, 3. , 3. , 3. , 3. , 3. , 0. , 0. , 0. , 0. , 0. ,
       0. , 1.5, 1.5, 1.5, 1.5, 1.5, 3. , 3. , 3. , 3. , 3. , 0. , 0. ,
       0. , 0. , 0. , 0. , 1.5, 1.5, 1.5, 1.5, 1.5, 3. , 3. , 3. , 3. ,
       3. , 0. , 0. , 0. , 0. , 0. , 1.5, 1.5, 1.5, 1.5, 1.5, 3. , 3. ,
       3. , 3. , 3. , 0. , 0. , 0. , 0. , 0. , 1.5, 1.5, 1.5, 1.5, 1.5,
       3. , 3. , 3. , 3. , 3. , 0. , 0. , 0. , 0. , 0. , 1.5, 1.5, 1.5,
       1.5, 1.5, 3. , 3. , 3. , 3. , 3. , 0. , 0. , 0. , 0. , 0. , 1.5,
       1.5, 1.5, 1.5, 1.5, 3. , 3. , 3. , 3. , 3. , 0. , 0. , 0. , 0. ,
       0. , 1.5, 1.5, 1.5, 1.5, 1.5, 3. , 3. , 3. , 3. , 3. , 0. , 0. ,
       0. , 0. , 0. , 1.5, 1.5, 1.5, 1.5, 1.5, 3. , 3. , 3. , 3. , 3. ,
       0. , 0. , 0. , 0. , 0. , 1.5, 1.5, 1.5, 1.5, 1.5, 3. , 3. , 3. ,
       3. , 3. , 0. , 0. , 0. , 0. , 0. , 1.5, 1.5, 1.5, 1.5, 1.

In [3]:
logger_init(log_file_name="lowerlimb_first_try")

In [ ]:
logging.shutdown()  # 关闭当前的日志处理器，解除文件占用、关闭后日志文件可删除

In [23]:
# 由于随机划分数据集导致模型分类结果有差异，因此重复运行10次取平均值
# 每次运行模型前需要
score_train_list = []
score_test_list = []
mse_train_list = []
mse_test_list = []

In [7]:
# 生成需要归一化的列名list
muscle_name_lst = ['RF', 'LF', 'SarT', 'BF', 'SemiT', 'VM']
feature_name_lst = ['initial_pressure_ave', 'FMG', 'mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']
col_lst = []
for muscle in muscle_name_lst:
    for fea in feature_name_lst:
        col_lst.append(muscle + fea)

len(col_lst)

54

In [26]:
'''
最传统做法
随机划分训练集和测试集，进行cv参数搜索
'''
# 选择动作类型，前抬腿or后抬腿
motion_data_df = data_df[(data_df['motion_type'] == 'hou') & (data_df['side'] == 'healthy')]

y_data = data_df.loc[:, 'label'].values
x_data = data_df.loc[:, col_lst].values

print(x_data.shape)
print(y_data.shape)

# 划分训练集和测试集
train_data, test_data, train_label, test_label = train_test_split(x_data, y_data, test_size = 0.2, random_state = 1, stratify = y_data)

"""
Grid searching best parameters of SVR model
"""
# 记录开始训练时间
start_time = time.perf_counter()

# 自动选择合适的参数
svr = GridSearchCV(SVR(kernel='rbf'), param_grid={"C": np.logspace(-3, 3, 7), "gamma": np.logspace(-3, 3, 7)}, n_jobs=-1)
svr.fit(train_data, train_label)
print(f"Best params: {svr.best_params_}")

end_time = time.perf_counter()
print(f"training time(min): {(end_time - start_time)/60}")

score_test = svr.score(test_data, test_label)
score_train = svr.score(train_data, train_label)
print(f"train score: {str(score_train)}")
print(f"test score: {str(score_test)}")
test_pre = svr.predict(test_data)
train_pre = svr.predict(train_data)
print(f"train mean squared error: {mean_squared_error(train_label, train_pre)}")
print(f"test mean squared error: {mean_squared_error(test_label, test_pre)}")

# 多次测试结果存入list方便计算平均值
score_train_list.append(score_train)
score_test_list.append(score_test)
mse_train_list.append(mean_squared_error(train_label, train_pre))
mse_test_list.append(mean_squared_error(test_label, test_pre))
print(len(score_test_list))

(306, 54)
(306,)
Best params: {'C': 10.0, 'gamma': 0.01}
training time(min): 0.005712646666628037
train score: 0.972072426198342
test score: 0.5784248758931846
train mean squared error: 0.04196770463953726
test mean squared error: 0.6425620843240977
3


In [22]:
# 合并结果并保存至csv
fuck_you_df = pd.DataFrame({'train_score': score_train_list,
                            'test_score': score_test_list,
                            'train_mse': mse_train_list,
                            'test_mse': mse_test_list})
df_save_csv(fuck_you_df, r"E:\fuckinggggg\paper_iFEMG_4th\data\fuck.csv")

File E:\fuckinggggg\paper_iFEMG_4th\data\fuck.csv saved!
